In [3]:
import sys
import os
import glob
import re
import numpy as np

# Keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Flask utils
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer


In [ ]:
model._make_predict_function()

In [ ]:
import sys
import os
import glob
import re
import numpy as np

# Keras
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import load_model
from keras.preprocessing import image

# Flask utils
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer

# Load trained model
model = load_model('model.h5')
model._make_predict_function()    # It create a function to run on GPU
print('Model loaded. Start serving...')




In [4]:
model = load_model('model.h5')
model._make_predict_function()    # It create a function to run on GPU
print('Model loaded. Start serving...')

labels_name = ['butterfly', 'cat', 'chicken', 'cow', 'dog', 'elephant', 'horse', 'sheep','spider','squirrel']


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Model loaded. Start serving...


In [20]:
img = image.load_img('dataset/validate/cat/34.jpeg',target_size=(200,200))
x= image.img_to_array(img)
x = np.expand_dims(x, axis=0) # (1,200,200,3)

In [23]:
z = model.predict(x)
labels_name[np.argmax(z)]

'butterfly'

In [ ]:
img = image.load_img(img_path, target_size=(200, 200, 3))
    x = image.img_to_array(img)  # (200,200,3)
    print('##########',x.shape)
    x = np.expand_dims(x, axis=0) # (1,200,200,3)
    print('#########',x.shape)
    preds = model.predict(x) # return (1,10)
    return labels_name[preds.argmax()]


@app.route('/predict', methods=['GET', 'POST'])
def upload():
    global model
    if request.method == 'POST':

        # Get the file from post request
        f = request.files['file']
        # Save the file to ./uploads
        basepath = ''#os.path.	dirname(__file__)
        file_path = os.path.join(basepath, 'uploads', secure_filename(f.filename))
        f.save(file_path)

        name =  model_predict(file_path, model)
        return name
    return 'None'


app.run()

In [ ]:
<FileStorage: '63.jpeg' ('image/jpeg')>

In [ ]:
#f = request.files['file']
#print(f)
        # Save the file to ./uploads
basepath = os.path.dirname(__file__)
        
#print(basepath)
#file_path = os.path.join(basepath, 'uploads', secure_filename(f.filename))
#print(file_path)

#f.save(file_path)


In [ ]:
from tensorflow.keras import models

In [ ]:
import h5py

# Animal Classifier

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import numpy as np
from datetime import datetime
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import math
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

import tensorflow.keras
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Sequential

In [ ]:
train_dir = 'dataset/train/'
test_dir = 'dataset/validate/'

# number of images in the training dataset
n_train = 8000

# number of images in the validation dataset
n_test = 2000

# the size of the image (h,w,c)
input_shape = (200,200, 3)

# size of each mini-batch
batch_size = 32

# nunmber of training episodes
epochs = 10

## Loading the image dataset in a way that is for efficient training
### Image Data Generators
A naive approach to data loading is to load all the images and transform them up front. This would result in a huge amount of used RAM before training starts. Your machine might not be able to handle this, which would result in crashing kernels. It can also take a very long time depending on the dataset.

Instead we can load and transform images required exactly when we need it. This would be when feeding a batch of images to the model during training. 

Keras provides an optimized method of doing this with the Image Data Generator class. It allows us to load images from a directory efficiently. These generators can also transform the dataset in many other ways to augment it.  Explore these optional transformations to help make your model more general, and improve accuracy.

In [ ]:
# define data generators
train_data_generator = ImageDataGenerator(rescale=1./255)
test_data_generator  = ImageDataGenerator(rescale=1./255)

# tell the data generators to use data from the train and validation directories
train_generator = train_data_generator.flow_from_directory(train_dir,
                                                          target_size=(200,200),
                                                          batch_size=batch_size,
                                                          class_mode='categorical')

test_generator = test_data_generator.flow_from_directory(test_dir,
                                                          target_size=(200,200),
                                                          batch_size=batch_size,
                                                          class_mode='categorical')

In [ ]:
# Checking what is in train_generator
a,b = next(train_generator)
a.shape,b.shape # it create 32 size of batches 

### Get Class Names

It is useful to have a dictionary of image classes. We can use this dictionary to make our predictions more human-readable.

In [ ]:
# get a dictionary of class names
labels_dictionary = train_generator.class_indices

# turn classes dictionary into a list
labels_name = list(labels_dictionary.keys())

# get the number of classes
n_labels = len(labels_name)

## Building the image classifier model
Our model consists of many layers. Images are passed through the model and a set of numbers are outputted. This set of numbers describe the probability of class the image is. We take the largest of these numbers as the most likely class. 

We will use several types of layers and activations:

1. `Conv2D` is a 2-dimensional convolutional layer. It applies filters over the inputted image. This helps the model learn about spatial relationships in the image.
2. `ReLu` is a type of non-linear activation function. It helps the model understand which neurons are activating.
3. `MaxPooling2D` downsamples its input. We use It to reduce the dimensionality of input. This creates a more abstract form of the input.
4. `Flatten` will turn a matrix into a row. Like flattening a muffin into a pancake. We use it so that we can feed the output into dense layers.
5. `Dense` is a densely-connected neural network layer. 
6. `Softmax` is an activation function. We use it to turn the output numbers into a range of 0 and 1. It will also cause all the outputted numbers to add up to 1. This can be interpreted as the decimal probability of a class.

Note that the last layer has the same number of neurons as classes. This means that this layer will output 10 numbers, mapping to a class.

In [ ]:
def create_model():
    # define the model 
    # takes in images, convoles them, flattens them, classifies them
    # input_shape = (200,200,3)
    
    model = Sequential()                                                                           
    model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu', padding='same', input_shape=input_shape))
    model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D())
    
    model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D())
    
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D())
    
    model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D())
    
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='sigmoid'))
    
    return model

In [ ]:
# define the loss function and optimizer
model = create_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# we try to find path of each image with labels so we directly feed them into training
def get_path_of_imgs(labels, directory):
    # Empty list which story the path of classes
    class_paths = {}
    for each_label in labels:
        image_paths = np.array([])
        # join  directory + subclass path
        class_path = os.path.join(directory, each_label)
        # return list of all images in class_path
        images = os.listdir(class_path)
        for image in images:
            # join class_path + each_image
            image_path = os.path.join(class_path, image)
            # append each images into image_paths
            image_paths = np.append(image_paths, image_path)
        # append each classes images into class_paths
        class_paths[each_label] = image_paths        
    
    return class_paths

In [ ]:
img_path_of_train = get_path_of_imgs(labels_name, train_dir)
img_path_of_test  = get_path_of_imgs(labels_name, test_dir)

#pd.DataFrame(img_path_of_train).head()

In [ ]:
labels_name

In [ ]:
y_cap = predict(img_path_of_train['butterfly'][0], model,input_shape)

In [ ]:
img = load_img(img_path_of_train['butterfly'][0], target_size=(200, 200, 3))
x = img_to_array(img) # 200,200,3
x = np.expand_dims(x, axis=0) # (1,200,200,3)
#x = preprocess_input(x, mode='caffe') # (1,200,200,3)
preds = model.predict(x)
np.argmax(preds)

In [ ]:
img_to_array(img).shape

In [ ]:
preds.argmax(axis=-1)[0][0][1])

In [ ]:
decode_predictions()

# Try to predict single classes before starting training

In [ ]:
def predict(image_paths, model, input_shape):
    images_arr = []
    # load images     
    for image_path in image_paths:
        # load image and turn into array
        image_arr = img_to_array(load_img(image_path, target_size=input_shape))

        # add the image_arr to the images_arr array
        images_arr.append(image_arr)
    # turn it into a numpy arrays so that it can be feed into the model as a batch
    images = np.array(images_arr)
    # make a predictions on the batch
    predictions = model.predict(images, batch_size=batch_size)
    return predictions

In [ ]:
# passing only butterfly images and check what model give us output
y_cap = predict(img_path_of_train['butterfly'], model,input_shape)

# predict accuracy
predict_acuracy = lambda x,y_cap : np.count_nonzero(np.argmax(y_cap,axis=1) == x)/len(y_cap)

# calling lambda funtion
single_class_accuracy = predict_acuracy(labels_dictionary['butterfly'],y_cap)

print('Accuracy before training when we feed only butterfly images: ',single_class_accuracy ,'%')

## Training the model 
This model has over 5,000,000 trainable parameter - far too many to set manually. We need to train the model with the training dataset so that the model can to learn the optimal weights that should be used. These weights are the parameter values of the model. 

In [ ]:
# log information for use with tensorboard
tensorboard = TensorBoard(log_dir='logs/')

In [ ]:
# train the model using the training data generator
history = model.fit_generator(train_generator,
                    steps_per_epoch = math.floor(n_train/batch_size),
                    validation_data=test_generator,
                    validation_steps=math.floor(n_test/batch_size),
                    epochs=5,
                    callbacks=[tensorboard])                                                                                

### Examine Model Accuracy After Some Training
Let’s examine how well the model performs now that we've trained it a bit. Again, we will determine the model’s accuracy on 1 class.

In [ ]:
# passing only butterfly images and check what model give us output
y_cap = predict(img_path_of_train['butterfly'], model, input_shape)

# calling lambda funtion
single_class_accuracy = predict_acuracy(labels_dictionary['butterfly'],y_cap)

print('Accuracy after two epochs when we feed only butterfly images: ',single_class_accuracy ,'%')

### Continue Training the Model
Let's continue training the model.

### Examine Model Accuracy After Training
Now that we've completed training the model, let's examine it's accuracy on 1 class.

In [ ]:
def plot_prediction(class_keys, image_paths, predictions):
    """
    Plots image predictions with the most likely class, and the probabilities of the prediction.
        
    class_keys: list of class keys
    image_paths: path to an image
    predictions: predictions of the image_paths
    """
        
    for index, image_path in enumerate(image_paths):
        # determine the most likely class from the prediction
        most_likely_class = np.argmax(predictions[index])

        # add class labels for the prediction
        # remember that we feed in a batch so we need to grab the first prediction
        prediction_classes = [str(class_keys[prob_index]) + ": " + str(round(prob*100, 4)) + "%" for prob_index, prob in enumerate(predictions[index])]

        # generate the prediction label
        subplot_label = "Prediction: " + str(class_keys[most_likely_class]) + "\nProbabilities: " + ', '.join(prediction_classes)

        # setup a plot
        fig = plt.figure(figsize=(7, 7), tight_layout=True)
        fig.set_facecolor('white')
        
        # load the image
        image_pil = load_img(image_path, interpolation='nearest', target_size=(200,200))

        # render an image to the plot
        ax = fig.add_subplot(1, 1, 1)
        ax.imshow(image_pil)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(subplot_label)

## Tensorboard

### Training Loss

![Training Loss](assets/loss.png "Training Loss")

### Training Accuracy
![Training Accuracy](assets/acc.png "Training Accuracy")

### Validation Loss
![Validation Loss](assets/val_loss.png "Validation Loss")

### Validation Accuracy
![Validation Accuracy](assets/val_acc.png "Validation Accuracy")

## Predict
It is useful to know which image predictions were correct and which were wrong. Let’s examine 10 predictions, 1 prediction per class.

In [ ]:
# get 1 image path per class
predict_image_paths = [img_path_of_train[image_path][0] for image_path in img_path_of_train]

In [ ]:
# Make 1 prediction per class
predictions = predict(predict_image_paths, model, input_shape)

# plot the image that was predicted
#plot_prediction(labels_name, predict_image_paths, predictions)

In [ ]:
# export the model for later
model.save('model.h5')